In [30]:
pip install googletrans==4.0.0-rc1

In [31]:
import imaplib
import email
from email.header import decode_header, make_header
from googletrans import Translator

In [32]:
######################IMAP을 통해 최근 메일을 읽는 프로그램(참고용)##########################


server = imaplib.IMAP4_SSL('imap.gmail.com') #IMAP4 서버에 암호화하여 접근 / IMAP4는 메일 서버에 저장되어있는 메일에 접근하기 위한 방법
server.login('hhj04021@likelion.org', '39064638') #내 계정 정보로 로그인하기 / 여기서 오류 발생 시 운영진에게 문의...

('OK', [b'hhj04021@likelion.org authenticated (Success)'])

In [33]:
rv, data = server.select() 
#select함수 = 메일 서버에 저장된 모든 메일 건수를 알려준다. 
#실행결과는 실행 성공 여부 & 메일 건수 두 정보 출력
#rv, data변수에 나눠담기, rv 출력 시 ok 출력, data 출력 시 메일 건수 출력, 우리는 메일 건수만 필요하니까 data만 사용할 것


print(rv)
print(data)


OK
[b'1']


In [34]:
recent_no = data[0] 
#가장 최근의 메시지 번호/최근의 메세지 번호는 메일의 건수와 동일하다.

rv, fetched = server.fetch(recent_no, '(RFC822)') 
#fetch함수에 두 가지 인자를 전달, 최근 메세지 번호와
#RFC822는 표준 메세지 형식 (중요X)
#fetch 실행결과는 아까처럼 두 가지 변수에 나누어 담았다.
#마찬가지로 rv는 fetch함수가 잘 실행되었다는 정보인 ok가 담김
#가져온 <최근 메일>의 모든 정보는 fetched 변수에 담김


message = email.message_from_bytes(fetched[0][1]) 
#메세지 내용에 해당하는 부분을 파싱하는 단계
#여기서 우리가 필요한 메일 내용은 fetched의 첫 번째 요소의 두 번째 요소라서 [0][1]로 표현함
#print(fetched[0][1])해보면 엄청 길고 못 알아보는 인자가 출력되는데 이 부분이 우리가 가져와야 할 메일 내용이라는 것만 이해해주시면 됩니다.
#email.message_from_bytes는 우리가 알아볼 수 있는 형태로 바꿔주는 함수
#변환한 내용을 message객체에 담았고 이제 이 객체를 이용해 이메일의 제목, 보낸이, 내용 등을 가져올 수 있게 됨.


fr = make_header(decode_header(message.get('From')))
subject = make_header(decode_header(message.get('Subject')))
#보낸이와 제목은 이메일의 헤더 항목입니다.
#따라서, 이를 정확한 값으로 가져오기 위해 decode_header, make_header함수를 이용했습니다.


In [35]:
'''메세지 본문 내용을 구하는 과정입니다. ('''
if message.is_multipart():
    for part in message.walk():
        ctype = part.get_content_type()
        cdispo = str(part.get('Content-Disposition'))
        if ctype == 'text/plain' and 'attachment' not in cdispo:
            body = part.get_payload(decode=True)
            break
else:
    body = message.get_payload(decode=True)

body = body.decode('utf-8')



print(f"보낸사람:{fr}")
print(f"제목:{subject}")


보낸사람:20200221@likelion.org
제목:아기사자님. 번역 부탁드립니다!


In [36]:
####################################5개 국어 번역기 프로그램##################################################

#1. 번역할 내용은 body변수에 담겨있습니다. 이를 f-string을 이용해 outsourcing에 담아주세요.
outsourcing = f'{body}'

#2. 번역할 내용을 출력해봅시다.
print(outsourcing)

안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.



In [37]:
#3. 번역기를 만들어주세요.
translator = Translator()

#4. 5개 국어의 언어 코드를 리스트에 담아주세요.
langlist = ['en','ja','zh-cn','es','fr']


#5. 5개 국어 각각에 대한 결과를 아래와 같이 출력하세요.
for dest in langlist:
  result = translator.translate(outsourcing, dest)
  detected = translator.detect(outsourcing)
  print("===========출 력 결 과===========")
  print(detected.lang,":",outsourcing)
  print(result.dest,":",result.text)
  print("=================================")

===========출 력 결 과===========
ko : 안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.

en : hello.Please translate what I wrote in English, Japanese, Chinese, Spanish, and French.
===========출 력 결 과===========
ko : 안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.

ja : こんにちは。英語、日本、中国語、スペイン語、フランス語で書いたものを翻訳してください。
===========출 력 결 과===========
ko : 안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.

zh-cn : 你好。请翻译我用英语，日语，中文，西班牙语和法语撰写的内容。
===========출 력 결 과===========
ko : 안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.

es : Hola.Traduzca lo que escribí en inglés, japonés, chino, español y francés.
===========출 력 결 과===========
ko : 안녕하세요. 제가 작성한 내용을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드립니다.

fr : salut.Veuillez traduire ce que j'ai écrit en anglais, japonais, chinois, espagnol et français.


In [38]:

'''
===========출 력 결 과===========
ko : 안녕하세요. 제 이름은 하연진입니다. 여기에 입력하는 내용들을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드려요.

en : hello.My name is Ha Yeonjin.Please translate the contents you enter here in English, Japanese, Chinese, Spanish, and French.
=================================
'''

'\n===========출 력 결 과===========\nko : 안녕하세요. 제 이름은 하연진입니다. 여기에 입력하는 내용들을 영어, 일본어, 중국어, 스페인어, 프랑스어로 번역 부탁드려요.\n\nen : hello.My name is Ha Yeonjin.Please translate the contents you enter here in English, Japanese, Chinese, Spanish, and French.\n=================================\n'

In [39]:
#6. "번역한 언어 : 번역한 결과" 형태로 5개 국어로 번역한 결과를 trans 변수에 담아주세요. (딕셔너리, 반복문 사용)
resultdict={}
for dest in langlist:
  result = translator.translate(outsourcing,dest)
  resultdict[result.dest]=result.text
print(resultdict)

trans = str(resultdict)

{'en': 'hello.Please translate what I wrote in English, Japanese, Chinese, Spanish, and French.', 'ja': 'こんにちは。英語、日本、中国語、スペイン語、フランス語で書いたものを翻訳してください。', 'zh-cn': '你好。请翻译我用英语，日语，中文，西班牙语和法语撰写的内容。', 'es': 'Hola.Traduzca lo que escribí en inglés, japonés, chino, español y francés.', 'fr': "salut.Veuillez traduire ce que j'ai écrit en anglais, japonais, chinois, espagnol et français."}


In [40]:
################################번역 결과 이메일 발송####################################


import smtplib #7. smtp메일서버로 메일을 쉽게 보내주는 라이브러리 import
from email.message import EmailMessage #8. 작성하고 싶은 메일 내용을 MIME타입으로 변환해주는 기능 import

'''1. 서버 정보 생성'''
SMTP_SERVER = "smtp.gmail.com" #9. 서버 정보
SMTP_PORT =465 #10. 포트 정보


'''2. MIME 형태의 메세지 통 만들고 번역결과를 메일 내용에 담기'''
message = EmailMessage() #11. 메세지 통을 생성해주세요.
message.set_content(trans) #12. 번역 결과를 메일 내용에 담아주세요.



In [41]:
'''3. 메일의 헤더 작성'''
message["Subject"]="황혜정_토요일팀_성공입니다~!" #13. 제목은 "이름_X요일팀_성공입니다~!"로 설정해주세요.
message["From"]= "hhj04021@likelion.org" #14. 발신자는 본인의 likelion.org 계정을 입력해주세요.
message["To"]=fr #15. 수신자는 앞에서 변수를 찾아서 입력해주세요. (이메일 직접 입력X)


'''4. SMTP서버에 연결하고 계정을 이용해 로그인'''
smtp = smtplib.SMTP_SSL(SMTP_SERVER,SMTP_PORT) #16. 암호화 방식의 smtp함수를 사용해 SMTP서버에 연결해주세요.


smtp.login('hhj04021@likelion.org', '39064638')


'''5. 메세지 전송, 전송 후 연결 끊기'''
smtp.send_message(message) #19. 메세지를 전송해주세요.
smtp.quit() #20. 서버와의 연결을 끊어주세요.


server.close()
server.logout()

('BYE', [b'LOGOUT Requested'])